In [1]:
from unsloth import FastLanguageModel
import torch, psutil, os, gc
from datasets import load_dataset, concatenate_datasets
from trl import SFTTrainer, SFTConfig
from datetime import datetime

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
# # 모델 스냅샷 다운로드
# from huggingface_hub import snapshot_download

# snapshot_download(
#     repo_id="naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B",
#     local_dir="../model/HyperCLOVAX-SEED-Text-Instruct-1.5B",
#     local_dir_use_symlinks=False    # 실제 파일 복사본을 만들기 위해 반드시 False
# )

In [2]:
model_name = "../model/LFM2.5-1.2B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2026.1.2: Fast Lfm2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj",
        "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = True
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
dataset1 = load_dataset("json", data_files="./data/train_data_singleturn.jsonl")
dataset2 = load_dataset("json", data_files="./data/train_data_function_call_lfm.jsonl")

dataset = concatenate_datasets([
    dataset1['train'],
    dataset2['train']
]).shuffle(seed=42)
dataset[0]

{'messages': [{'role': 'user',
   'content': '카드 지갑을 잃어버려서 농협은행 서대문본점 가야 해. 주소 좀 가르쳐줘.'},
  {'role': 'assistant',
   'content': '<|tool_call_start|>[search_address(location="농협은행 서대문본점")]<|tool_call_end|>'},
  {'role': 'tool',
   'content': '{"location":"농협은행 서대문본점","address":"서울특별시 중구 통일로 120"}'},
  {'role': 'assistant',
   'content': '지갑을 잃어버리셔서 많이 당황하셨겠어요. 농협은행 서대문본점 주소는 서울특별시 중구 통일로 120입니다. 빨리 처리되어서 걱정을 덜으셨으면 좋겠네요.'}]}

In [5]:
# 도구 함수 정의
def get_current_time():
    now = datetime.now()
    hour = now.hour
    minute = now.minute
    second = now.second
    return {
        "hour": hour,
        "minute": minute,
        "second": second
    }

def get_current_date():
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day
    day_of_week = now.strftime("%A")
    return {
        "year": year,
        "month": month,
        "day": day,
        "day_of_week": day_of_week
    }

def get_weather(city: str):
    """날씨 정보 (더미)"""
    # 실제로는 API 호출
    return {
        "city": city,
        "weather": "맑음",
        "temperature": 25,
        "humidity": 60
    }

def search_address(location):
    """주소 검색 (더미)"""
    return {
        "location": location,
        "address": "서울시 강남구 27번길"
    }

# 함수 매핑
TOOLS = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_weather": get_weather,
    "search_address": search_address,
}

# 2️⃣ Tools 정의 (Kanana에 전달)
TOOL_DEFINITIONS = [
    {
        "name": "get_current_time",
        "description": "현재 시간을 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_date",
        "description": "현재 날짜를 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_weather",
        "description": "특정 지역의 날씨 정보를 조회합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "지역 이름"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_address",
        "description": "주소를 검색합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "검색어"
                }
            },
            "required": ["location"]
        }
    }
]

In [ ]:
# # assistant_only_loss = False일 경우
# def formatting_prompts_func(examples):
#     texts = []

#     for message in examples['messages']:
#         text = tokenizer.apply_chat_template(
#             message,
#             tokenize=False,
#             add_generation_prompt=False,
#             tools=TOOL_DEFINITIONS
#         )

#         texts.append(text)

#     return {'text': texts}

# formatted_dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)
# print(formatted_dataset[0])

# split_dataset = formatted_dataset.train_test_split(test_size=0.1)
# train_ds = split_dataset["train"]
# eval_ds = split_dataset["test"]

{'text': '<|startoftext|><|im_start|>system\nList of tools: [{"name": "get_current_time", "description": "현재 시간을 반환합니다", "parameters": {"type": "object", "properties": {}, "required": []}}, {"name": "get_current_date", "description": "현재 날짜를 반환합니다", "parameters": {"type": "object", "properties": {}, "required": []}}, {"name": "get_weather", "description": "특정 지역의 날씨 정보를 조회합니다", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "지역 이름"}}, "required": ["city"]}}, {"name": "search_address", "description": "주소를 검색합니다", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "검색어"}}, "required": ["location"]}}]<|im_end|>\n<|im_start|>user\n카드 지갑을 잃어버려서 농협은행 서대문본점 가야 해. 주소 좀 가르쳐줘.<|im_end|>\n<|im_start|>assistant\n<|tool_call_start|>[search_address(location="농협은행 서대문본점")]<|tool_call_end|><|im_end|>\n<|im_start|>tool\n{"location":"농협은행 서대문본점","address":"서울특별시 중구 통일로 120"}<|im_end|>\n<|im_start|>assistant\n지갑을 잃어버리셔서

In [7]:
# assistant_only_loss = True일 경우
def formatting_prompts_func(examples):
    if len(examples["messages"]) > 0 and isinstance(examples["messages"][0], list):
        messages_list = examples["messages"] # 예) {"messages": [ [채팅1], [채팅2], [채팅3] ]}
    else:
        # 단일 샘플 처리 (SFTTrainer 검증용) 
        messages_list = [examples["messages"]] # 예) {"messages": [채팅1]}

    texts = []
    for message in messages_list:
        text = tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=False,
            tools=TOOL_DEFINITIONS
        )
        texts.append(text)

    # SFTTrainer의 formatting_func는 반드시 '문장들의 리스트'를 반환해야 한다.
    return texts

split_dataset2 = dataset.train_test_split(test_size=0.1)
train_ds2 = split_dataset2["train"]
eval_ds2 = split_dataset2["test"]

In [ ]:
args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 400,
        num_train_epochs=1,
        learning_rate = 5e-5, 
        bf16 = True,
        seed = 1234,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        output_dir = "outputs",

        logging_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,

        # SFTConfig의 파라미터
        packing=False,
        dataset_text_field="text",
        # model_init_kwargs={"trust_remote_code": True},
        assistant_only_loss=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds2,
    eval_dataset=eval_ds2,
    processing_class=tokenizer,
    formatting_func=formatting_prompts_func,
    args=args
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4050 [00:00<?, ? examples/s]

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/450 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [9]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,050 | Num Epochs = 1 | Total steps = 507
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 884,736 of 1,171,225,344 (0.08% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,2.330500,1.143022
200,0.744800,0.957420
300,0.636000,0.884140
400,0.594000,0.859726
500,0.568500,0.863245


Unsloth: Not an error, but Lfm2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=507, training_loss=0.968840162194457, metrics={'train_runtime': 647.2425, 'train_samples_per_second': 6.257, 'train_steps_per_second': 0.783, 'total_flos': 7217205906663936.0, 'train_loss': 0.968840162194457, 'epoch': 1.0})

### 테스트

In [1]:
import torch, psutil, os, gc, re, json
from datetime import datetime
from unsloth import FastLanguageModel
from transformers import GenerationConfig, TextIteratorStreamer
from peft import PeftModel
from threading import Thread

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
def print_mem(tag):
    process = psutil.Process(os.getpid())
    ram = process.memory_full_info().rss / 1024**2
    vram = torch.cuda.memory_allocated() / 1024**2
    peak = torch.cuda.max_memory_allocated() / 1024**2
    print(f"[{tag}] RAM: {ram:.1f} MB | VRAM: {vram:.1f} MB, VRAM peak: {peak:.1f} MB")

torch.cuda.empty_cache()
gc.collect()
print_mem("before")

# 모델 로드
model_name = "../model/LFM2.5-1.2B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)

model = PeftModel.from_pretrained(model, "./outputs/checkpoint-500")

# 추론 모드로 전환
FastLanguageModel.for_inference(model)

print_mem("after_load")

# 1 token 생성
prompt = "안녕하세요."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
_ = model.generate(**inputs, max_new_tokens=1)
print_mem("after_gen")

[before] RAM: 993.1 MB | VRAM: 0.0 MB, VRAM peak: 0.0 MB
==((====))==  Unsloth 2026.1.2: Fast Lfm2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
[after_load] RAM: 1186.1 MB | VRAM: 771.5 MB, VRAM peak: 1108.1 MB
[after_gen] RAM: 1717.3 MB | VRAM: 779.6 MB, VRAM peak: 1108.1 MB


In [3]:
# 도구 함수 정의
def get_current_time():
    now = datetime.now()
    hour = now.hour
    minute = now.minute
    second = now.second
    return {
        "hour": hour,
        "minute": minute,
        "second": second
    }

def get_current_date():
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day
    day_of_week = now.strftime("%A")
    return {
        "year": year,
        "month": month,
        "day": day,
        "day_of_week": day_of_week
    }

def get_weather(city: str):
    """날씨 정보 (더미)"""
    # 실제로는 API 호출
    return {
        "city": city,
        "weather": "맑음",
        "temperature": 25,
        "humidity": 60
    }

def search_address(location):
    """주소 검색 (더미)"""
    return {
        "location": location,
        "address": "서울시 강남구 27번길"
    }

# 함수 매핑
TOOLS = {
    "get_current_time": get_current_time,
    "get_current_date": get_current_date,
    "get_weather": get_weather,
    "search_address": search_address,
}

# 2️⃣ Tools 정의 (Kanana에 전달)
TOOL_DEFINITIONS = [
    {
        "name": "get_current_time",
        "description": "현재 시간을 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_date",
        "description": "현재 날짜를 반환합니다",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_weather",
        "description": "특정 지역의 날씨 정보를 조회합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "지역 이름"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_address",
        "description": "주소를 검색합니다",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "검색어"
                }
            },
            "required": ["location"]
        }
    }
]

In [4]:
generation_config = model.generation_config
generation_config.max_new_tokens = 512
generation_config.temperature = 0.6
generation_config.top_p = 0.95
generation_config.do_sample = False
generation_config.use_cache = False
generation_config.repetition_penalty=1.2

def stream_with_langchain(question):
    messages = [
        {"role": "system", "content": f"List of tools: {json.dumps(TOOL_DEFINITIONS)}"},
        {"role": "user", "content": question}
    ]
    
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(
        formatted_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024,
    )

    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    
    streamer = TextIteratorStreamer(
        tokenizer,
        skip_special_tokens=True,   # <|end|> 같은 특수 토큰 건너뛰기
        skip_prompt=True)           # 입력 프롬프트 건너뛰기 (응답만 보기)
    
    # Thread가 있어야 메인 스레드에서 백그라운드로 generate()가 실행되고
    # 서브 스레드에서 streamer로 토큰을 실시간으로 읽을 수 있음
    # 따라서 동시에 병렬로 처리 가능

    thread = Thread(
        target=model.generate, 
        kwargs=dict(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
            streamer=streamer,
        )
    )
    thread.start()
    
    full_response = ""
    for text in streamer:
        print(text, end="", flush=True)
        full_response += text
    
    thread.join()
    print()     # 줄바꿈

    pattern = r"<\|tool_call_start\|>\[(.*?)\]<\|tool_call_end\|>"
    match = re.search(pattern, full_response)

    if match:
        call = match.group(1) # get_weather(location="Seoul")
        func_name, arg_str = call.split("(", 1)
        arg_str = arg_str.rstrip(")")

        args = {}
        if arg_str.strip():
            for part in arg_str.split(","):
                k, v = part.split("=")
                args[k.strip()] = v.strip().strip('"')

        tool_result = TOOLS[func_name](**args)

        messages.append({
            "role": "assistant",
            "content": full_response
        })
        messages.append({
            "role": "tool",
            "content": json.dumps(tool_result, ensure_ascii=False)
        })

        formatted_text_2 = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        inputs2 = tokenizer(
            formatted_text_2,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024,
        )

        streamer2 = TextIteratorStreamer(
            tokenizer,
            skip_special_tokens=True,   # <|end|> 같은 특수 토큰 건너뛰기
            skip_prompt=True            # 입력 프롬프트 건너뛰기 (응답만 보기)
        )

        thread2 = Thread(
            target=model.generate, 
            kwargs=dict(
                input_ids=inputs2["input_ids"].to(model.device),
                attention_mask=inputs2["attention_mask"].to(model.device),
                generation_config=generation_config,
                streamer=streamer2,
            )
        )
        thread2.start()

        final_response = ""
        for text in streamer2:
            print(text, end="", flush=True)
            final_response += text

        thread2.join()
        print()

        return final_response

    return full_response

In [5]:
question = "너는 누구야?"
print(f"Q: {question}")
print("A: ", end="", flush=True)
response = stream_with_langchain(question)

Q: 너는 누구야?
A: 저는 오늘 아침에 일어나서 커피를 마시면서 주변을 살펴보는 게 제일 좋아요.


### GGUF 변환

In [ ]:
from unsloth import FastLanguageModel

# 모델 및 아답터 로드 (한 번에 불러오기)
adapter_path = "./outputs/kanana-tools-checkpoint-500" # 학습된 아답터

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = adapter_path, # 아답터 경로를 넣으면 베이스 모델까지 자동으로 찾아서 합친다.
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

# 젯슨용 GGUF로 저장하기
# q4_k_m 방식은 8GB 램 환경에서 성능 손실이 적으면서 용량이 매우 착함
model.save_pretrained_merged(
    "kanana_merged_temp",    # 저장될 폴더명/파일명
    tokenizer,
    save_method = "merged_16bit"
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.2 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


TypeError: unsloth_generic_save_pretrained_merged() got an unexpected keyword argument 'quantization_method'